In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import os
from keras import models, layers
from keras.preprocessing import image
from sklearn.model_selection import train_test_split
from keras.applications import Xception
from keras import optimizers, callbacks

import warnings
warnings.filterwarnings('ignore')

In [ ]:
os.chdir('../input/aptos2019-blindness-detection/')

In [ ]:
os.listdir()

In [ ]:
train_df = pd.read_csv('train.csv')

In [ ]:
train_df.head()

In [ ]:
# create img path
img_path_impute = lambda x : x + '.png'

In [ ]:
img_path_impute(train_df.loc[0, 'id_code'])

In [ ]:
train_df['img_path'] = train_df.loc[:, 'id_code'].apply(img_path_impute)

In [ ]:
train_df.head()

In [ ]:
sns.countplot('diagnosis', data = train_df)

In [ ]:
# img_path = f'train_images/{train_df['id_code']}'

img_test = plt.imread('test_images/270a532df702.png')
plt.imshow(img_test)

In [ ]:
# split train test
x = train_df['img_path']
y = train_df['diagnosis']

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state=1228, stratify=y)

In [ ]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
train_full = pd.concat([X_train, y_train], axis=1)
test_full = pd.concat([X_test, y_test], axis=1)

train_full['diagnosis'] = train_full['diagnosis'].astype('str')
test_full['diagnosis'] = test_full['diagnosis'].astype('str')

In [ ]:
train_full.head()

In [ ]:
train_full.info()

In [ ]:
# checker 
train_df[train_df.index == 2225]

In [ ]:
train_gen = image.ImageDataGenerator(rescale = 1./255,
                                     rotation_range=0.2,
                                     shear_range=0.2,
                                     horizontal_flip=True,
                                     vertical_flip=True,
                                     width_shift_range=0.2,
                                     height_shift_range = 0.2)

test_gen = image.ImageDataGenerator(rescale = 1./255)

In [ ]:
os.listdir()

In [ ]:
len(os.listdir('train_images'))

In [ ]:
img_dir = 'train_images/'
train_data = train_gen.flow_from_dataframe(train_full, directory = img_dir, x_col = 'img_path', y_col = 'diagnosis', target_size=(299, 299), class_mode='categorical', batch_size=32)

test_data = test_gen.flow_from_dataframe(test_full, directory = img_dir, x_col = 'img_path', y_col = 'diagnosis', target_size=(299,299), class_mode='categorical', batch_size=32)



In [ ]:
train_step = train_data.n // train_data.batch_size
test_step = test_data.n // test_data.batch_size

In [ ]:
train_data.class_indices

In [ ]:
counter = 0
for batch in train_data:
    img_batch = batch[0]
    for img in range(img_batch.shape[0]):
        plt.figure()
        plt.imshow(img_batch[img])
        
    counter += 1
    if counter == 1:
        break

In [ ]:
xception = Xception(include_top =False, input_shape=(299,299,3))

In [ ]:
model = models.Sequential()
model.add(xception)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(5, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
my_opt = optimizers.nadam(lr=0.01)
call_early_stop = callbacks.EarlyStopping(monitor='val_loss', patience=3)
call_reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=5, min_lr = 1e-9, verbose=1)
call_model_save = callbacks.ModelCheckpoint(filepath = 'test_model.h5', save_best_only=True, verbose=1)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=my_opt, metrics=['acc'])

In [ ]:
history_v1 = model.fit_generator(train_data, 
                    steps_per_epoch=train_step, 
                    epochs=100, 
                    validation_data=test_data,
                    validation_steps=test_step, 
                    callbacks=[call_reduce_lr])

In [ ]:
model = models.Sequential()
model.add(xception)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(1024, activation='relu'))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=my_opt, metrics=['acc'])

In [ ]:
history_v1 = model.fit_generator(train_data, 
                    steps_per_epoch=train_step, 
                    epochs=100, 
                    validation_data=test_data,
                    validation_steps=test_step, 
                    callbacks=[call_reduce_lr])

In [ ]:
# display image from kernel : densenet-keras-starter
def display_samples(df, columns=4, rows=3):
    fig=plt.figure(figsize=(5*columns, 4*rows))

    for i in range(columns*rows):
        image_path = df.loc[i,'id_code']
        image_id = df.loc[i,'diagnosis']
        img = cv2.imread(f'../input/aptos2019-blindness-detection/train_images/{image_path}.png')
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        fig.add_subplot(rows, columns, i+1)
        plt.title(image_id)
        plt.imshow(img)
    
    plt.tight_layout()

display_samples(train_df)